In [1]:
import numpy as np
import matplotlib.pyplot as plt
import deepdish as dd

### load data

In [4]:
vector_field = dd.io.load('vector_fields.h5')

In [5]:
vector_field.shape

(105, 4, 840, 840)

### create abs distance matrices

In [6]:
abs_distance = np.zeros_like(vector_field)

### 4 channels meaning: (dxh, dyh, dxt, dyt)

In [7]:
abs_distance.shape

(105, 4, 840, 840)

### compute abs distance

In [22]:
def compute_abs_dist(abs_distance):
    pad_distance = np.arange(abs_distance.shape[2])
    n_repeat = abs_distance.shape[2]
    n_data = abs_distance.shape[0]
    
    for i in range(n_data):
        abs_distance[i][1] = abs_distance[i][3] = np.repeat(pad_distance.reshape(-1,1), n_repeat, axis=1)
        abs_distance[i][0] = abs_distance[i][2] = np.repeat(pad_distance.reshape(1,-1), n_repeat, axis=0)
    
    return abs_distance

In [23]:
abs_distance = compute_abs_dist(abs_distance)

### add abs distance to vector field

In [24]:
abs_vf = vector_field + abs_distance

### compare abs_vf location with labels in labeled data e.g. labeled_data[0] and abs_vf[0]

In [86]:
labeled_data = dd.io.load('clean_labeled_data.h5')

In [87]:
labeled_data.shape

(105, 840, 840)

In [229]:
# get abs_vf key point indeces
def get_indeces(matrix):
    n_channel = matrix.shape[0]
    x_indeces = []
    y_indeces = []
    
    x_indeces.append(np.concatenate((np.unique(matrix[0]), np.unique(matrix[2]))))
    y_indeces.append(np.concatenate((np.unique(matrix[1]), np.unique(matrix[3]))))
    
    return np.unique(np.array(x_indeces).reshape(1,-1)).astype(np.int64), np.unique(np.array(y_indeces).reshape(1,-1)).astype(np.int64)

In [230]:
get_indeces(abs_vf[30])

(array([309, 389, 630, 709, 724, 761, 778]),
 array([108, 110, 173, 421, 435, 445, 514]))

In [206]:
labeled_indeces = np.where(labeled_data[30]!=0)

In [128]:
x_indeces, y_indeces = get_indeces(abs_vf[0])

In [191]:
#x_indeces, y_indeces = get_indeces(abs_vf[30])

In [204]:
np.unique(abs_vf[30][0])

array([ 389.,  709.,  761.])

In [209]:
np.unique(abs_vf[30][2])

array([ 309.,  630.,  724.,  778.])

In [211]:
np.unique(abs_vf[30][1])

array([ 108.,  110.,  435.,  514.])

In [210]:
np.unique(abs_vf[30][3])

array([ 108.,  173.,  421.,  445.])

In [215]:
np.unique(labeled_indeces[0])

array([309, 389, 630, 709, 724, 761, 778])

In [200]:
np.setdiff1d(labeled_indeces[0], np.unique(x_indeces)).size

16

### verify if abs+vector_field is equivelent to coordinates of key_points for origional dataset

In [235]:
def verify_identical(labeled_data, abs_vf):
    n_data = len(abs_vf)
    n_error = 0
    
    for i in range(n_data):
        labeled_indeces = np.where(labeled_data[i]!=0)
        x_indeces, y_indeces = get_indeces(abs_vf[i])
        if np.setdiff1d(np.unique(labeled_indeces[0]), x_indeces).size!=0 or np.setdiff1d(np.unique(labeled_indeces[1]), y_indeces).size!=0:
            print(i)
            n_error += 1
    
    if n_error == 0:
        print('all abs+vf are identical to labeled_indeces!\n')

In [236]:
verify_identical(labeled_data, abs_vf)

all abs+vf are identical to labeled_indeces!



In [160]:
np.setdiff1d(labeled_indeces[0], x_indeces).size

0

In [161]:
np.setdiff1d(labeled_indeces[1], y_indeces).size

0

In [163]:
if np.setdiff1d(labeled_indeces[0], x_indeces).size!=0 or np.setdiff1d(labeled_indeces[1], y_indeces).size!=0:
    print("hello")

In [177]:
np.where(labeled_data[0]!=0)

(array([527, 612, 622, 625, 626, 634, 695, 707]),
 array([304, 317, 454, 648, 527, 534, 605, 524]))

In [222]:
a = np.array([1,2,3])
b = np.array([9,8,7,6])
c = np.array([9,8,7,6,5,4])

In [228]:
np.concatenate((a,b,c))

array([1, 2, 3, 9, 8, 7, 6, 9, 8, 7, 6, 5, 4])